[Link to this document's Jupyter Notebook](HW2-Affine_transform.ipynb)

In order to successfully complete this assignment, you must follow all the instructions in this notebook and upload your edited ipynb file to [D2L](http://d2l.msu.edu/) with your answers on or before **11:59pm on Friday February 19th**.

**BIG HINT:** Read the entire homework before starting.  

# Homework 2: Affine Transform

<img src="https://lh5.googleusercontent.com/T7SNdiXctkIi2iWHjjdAZCRBASzRjysFkmivxtD8cZ80QlKFPjOeYa--uiDoHEUdb7daadZQ-fW-l0SVBQJDj90vPFaCKke9JSf6jf8Rk71zpy62TP_hmBpdFONBoA=w740" alt="An figure showing how we will be stitching images together"> 

In this homework you will use the ```ipywidgets.interact``` (or ```ipywidgets.interact_manual```) function to build a interface to translate, scale, rotate and shear an image using ```skimage.transform``` and ```skimage.transform.warp``` (http://scikit-image.org/).  We will then "stitch" the images together similar to the following:




Here is a video of what your final solution may look like:

* Direct Link: https://youtu.be/b4x39uqTMBY

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("b4x39uqTMBY",width=640,height=360)

In [ ]:
# Here are some libraries you may need to use
%matplotlib inline
import matplotlib.pylab as plt

import sympy as sp
sp.init_printing()

import numpy as np

from ipywidgets import interact, interact_manual
import math


# Step 1: Read in image files

The following code reads a list of image files from a websites and saves them in their own 3D numpy arrays.  The first dimension of each array is the row, the second dimension is the column and the third dimension is the color (R,G,B). 


**NOTE (this is important to remember):** images have a coordinate system with origin in the upper left and the $z$ axis going into the screen. Also images are indexed by [Rows] and then [Columns] this means that the first index corresponds to the traditional y-axis and the second index corresponds to the x-axis (which is backwards to traditional euclidean axes).   

In [ ]:
#The following code snip-it downloads the files from internet and saves tnhem to your local directory.
from urllib.request import urlopen, urlretrieve
import imageio

#Load a list of images
ImageList = [('beaumont', 'http://res.cloudinary.com/miles-extranet-dev/image/upload/ar_16:9,c_fill,w_1000,g_face,q_50/Michigan/migration_photos/G21696/G21696-msubeaumonttower01.jpg'
),
             ('sparty','https://cdn.shopify.com/s/files/1/1058/4992/products/MSU-Gruffy-Sparty-Sticker-550_620x.jpg?v=1525615105'),
             ('billboard', 'https://lh4.googleusercontent.com/OA8Et4hB-vpuYTKt86gk9i_Or1ptNbxtuYnR-7vujPVuVx6m30wXoVE3_d5fG_xDYAxUmDxOUw=w600'),
             ('banner', 'https://lh4.googleusercontent.com/aORranhBKm_pVs-Mg_pkyY1e8ujLh_7Y671d2BjNrWPiagBqktcJvD34ZHgUQrgbFAj4xOcf0Q=w740')
            ]

for name, url in ImageList:
    print(name)
    file = name+'.png'
    urlretrieve(url, file);
    exec(name + " = imageio.imread(file)")

In [ ]:
#Pick an image and show it
im = banner
plt.imshow(im);

# STEP 2: Design Transforms

The skimage library allows us to apply a $3 \times 3$ transformation matrix to an image using the ```transform.warp``` function.

```warp```  uses an ```Inverse coordinate map``` as an input, which transforms coordinates in the output images ($P_o$) into their corresponding coordinates in the input image ($P_i$). This means we need to think about what we want the output image to look like and apply all of the transformations to get it to the original input image.


$$P_i = TP_o$$

Where $P_o$ is a point in the output image, $P_i$ is a point in the input image and $T$ is the $3x3$ image transform.


For example, if we want to have our output image shrink by a half this typically would be a contraction:

$$A = \left[ \begin{matrix}0.5 & 0 & 0 \\ 0 & 0.5 & 0 \\ 0 & 0 & 1 \end{matrix}\right]$$

However, warp wants transforms that go from output image to input image. This means we want to double the size of the image which can be done with a dilation:

$$B = \left[ \begin{matrix}2 & 0 & 0 \\ 0 & 2 & 0 \\ 0 & 0 & 1 \end{matrix}\right]$$

Also note that $A^{-1} = B$.

Here is an example of how to use the ```transform.warp``` function:

In [ ]:
from skimage import transform

S = np.matrix([[2,0,0], [0,2,0], [0,0,1]])
plt.imshow(transform.warp(im,S));
sp.Matrix(S)

What we really want is the output image to be centered. Since most transformations are symmetric around the origin it means we want to move the center of the image to the origin first (at least for scaling, rotation and shear).  In our scaling example, to go from a centered output we would first translate it from the center to the origin and then scale the image and translate it back to the center using the following transforms:

In [ ]:
# Translate image to put the center at the origin
Tc = np.matrix([[1,0,-im.shape[1]/2], [0,1,-im.shape[0]/2], [0,0,1]])
sp.Matrix(Tc)

In [ ]:
# translate image back so that the upper left point is at the origin

To = np.matrix([[1,0,im.shape[1]/2], [0,1,im.shape[0]/2], [0,0,1]])
sp.Matrix(To)

In [ ]:
#Combine the transforms into one matrix
T = To*S*Tc
sp.Matrix(T)

In [ ]:
plt.imshow(transform.warp(im, T));

&#9989;  **<font color=red>DO THIS:</font>** Construct another transformation matrix (named ```R```) that rotates the image 30 degrees (clockwise) around it's center.  Apply this new matrix such that we get a scaled and rotated image T.  Show the image and make sure it has in fact rotated by 30 degrees in the clockwise direction.

In [ ]:
#Put your rotation code here.

In [ ]:
from answercheck import checkanswer

checkanswer.matrix(R,"84b1a7ebd71e740c41730448de079684");

&#9989;  **<font color=red>DO THIS:</font>** write a third transformation matrix (named ```SH```) that conducts a shear of the image. The matrix should be in the following form with. To test this matrix use shx=-0.3 and shy=0.1:

$$A = \left[ \begin{matrix}1 & shx & 0 \\ shy & 1 & 0 \\ 0 & 0 & 1 \end{matrix}\right]$$

Apply this new matrix such that we get a scaled and sheared image.  Show the image. 


**Note:** the term shear may be new. here is a good matrix definition: https://www.mathworks.com/discovery/affine-transformation.html


In [ ]:
#Put your shear code here.

In [ ]:
#Check the Combined R and S matrix
from answercheck import checkanswer
checkanswer.matrix(SH,"b1259119454dbd7349c4ed47c1dea64a");

Lets use this technique to "stitch" two images together. This is a little tricky so we will use the following function (You do not need to modify this function). This function takes inputs; the background image, the top (warped image) and the corresponding transform needed to warp the image::

In [ ]:
def stitch_images(background, forground, T):
    """Function to stitch to images together by transforming the forground image by T 
    and then overlaying the resulting image on top of the background
    
    background: the background image to use
    forground: the image to be transformed
    T: 3x3 transformation matrix
    
    output: stitched image the same size as the background.
    
    """
    
    warpped_img = transform.warp(forground, T, output_shape=background.shape)
    
    if background.shape != warpped_img.shape:
        raise Exception('Size Missmatch Error - Input images should be the same size')
    
    if (len(background.shape) != 3):
        raise Exception('Image Type error - Images should be in RGB format')
            
    red_src = np.array(background[:,:,0])
    green_src = np.array(background[:,:,1])
    blue_src = np.array(background[:,:,2])

    if (np.max(warpped_img) == 1):
        scale = 255.0
        
    red_dst = warpped_img[:,:,0]*scale
    green_dst = warpped_img[:,:,1]*scale
    blue_dst = warpped_img[:,:,2]*scale

    image_mask = (np.sum(warpped_img, axis=2) == 0)
    
    red_new = red_src*image_mask
    green_new = green_src*image_mask
    blue_new = blue_src*image_mask
    
    im_new = np.zeros(background.shape)
    im_new[:,:,0] = (red_new + red_dst)/255
    im_new[:,:,1] = (green_new + green_dst)/255
    im_new[:,:,2] = (blue_new + blue_dst)/255
    
    return im_new

We can test the above function using the results of your previous transforms:

In [ ]:
Tc = np.matrix([[1,0,-billboard.shape[1]/2], [0,1,-billboard.shape[0]/2], [0,0,1]])
To = np.matrix([[1,0,banner.shape[1]/2], [0,1,banner.shape[0]/2], [0,0,1]])
plt.imshow(stitch_images(billboard, banner, To*R*S*SH*Tc))

# STEP 3: Build an combined affine transform function.

In this step, you will write a function that takes affine parameters as inputs and returns a combined $3 \times 3$ matrix. 

&#9989;  **<font color=red>DO THIS:</font>** Write a function that takes seven inputs and returns a transformation T. Name this function ```affine_transform```. These include the following :

- **a** - Rotation around the center of the image [-$180$,$180$].  
- **sx** - Sale factor for the image in the x direction (0,5]
- **sy** - Sale factor for the image in the x direction (0,5]
- **tx** - translation in the x direction [0, im.shape[1]]
- **ty** - translation in the y direction [0, im.shape[0]]
- **shx** - Shear in the x direction [0,1]
- **shy** - Shear in the y direction [0,1]

The order of these is vary important. First assume the image is already centered, apply shear,  then scaling, then rotation and then finish with translation. 

In [ ]:
##Put your function here
def affine_transform(a=0,sx=0.5, sy=0.5,tx=0,ty=0,shx=0, shy=0):
    T = np.matrix([[1,0,0],[0,1,0],[0,0,1]])
    #Your transformations go here.
    return T

In [ ]:
T = affine_transform()
T

In [ ]:
To*R*S*SH*Tc

# Step 4: Build an interactive interface
In this final step use the ```ipywidgets.interact``` (or ```ipywidgets.interact_manual```) function to call your ```affine_transform``` function turn all of the input parameters into sliders. 

**_Note_**: if the ```affine_transform``` function was created correctly in Step 3 then the following function do not need to be changed and should work similar to the video at the beginning of this homework. 

In [ ]:
forground = banner
background = billboard

def transform_and_plot(a=0,sx=0.5,sy=0.5, tx=0,ty=0, shx=0, shy=0):
    T = affine_transform(a=a,sx=sx, sy=sy,tx=tx,ty=ty,shx=shx, shy=shy)
    Tc = np.matrix([[1,0,-background.shape[1]/2], [0,1,-background.shape[0]/2], [0,0,1]])
    To = np.matrix([[1,0,forground.shape[1]/2], [0,1,forground.shape[0]/2], [0,0,1]])
    im_new = stitch_images(background, forground, To*T*Tc)
    plt.imshow(im_new);
    plt.show(); 


In [ ]:
interact(transform_and_plot, a1=(-180,180), sx=(0.001,5, 0.01), sy=(0.001,5, 0.01), tx=(-1,1,0.01), ty=(-1,1,0.01), shx = (-1,1,0.01), shy = (-1,1,0.01)); ##TODO: Modify this line of code

&#9989;  **<font color=red>DO THIS:</font>** Test your sliders and make sure they make sense. If the output differs slightly from the example video, explain what is different and why your solution also makes sense.

<font size=8 color="#009600">&#9998;</font> Do This - Erase the contents of this cell and replace it with your answer to the above question!  (double-click on this text to edit this cell, and hit shift+enter to save the text)

# Step 5: Make a billboard:


In [ ]:
T = affine_transform(a=0, 
                     sx=1,
                     sy=1,
                     tx=0,
                     ty=0,
                     shx=0,
                     shy=0)

im_new = stitch_images(billboard, banner, T)
plt.imshow(im_new);

# Step 6: Be creative

Find two other examples images (include their URLs in the list at the top of this assignment).  Stitch them together in a fun and creative way and show your final results.

---------
### Congratulations, we're done!

Turn in your assignment using D2L no later than **_11:59pm on the day of class_**. See links at the end of this document for access to the class timeline for your section.

Written by Dirk Colbry, Michigan State University
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.